## 准备数据

In [15]:
import os
import tensorflow as tf
from tensorflow.python.keras import optimizers
import tensorflow_datasets as tfds
import numpy as np

os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}


def mnist_dataset():
    datasets = tfds.load(
        "mnist", split=["train", "test"], shuffle_files=True, as_supervised=True
    )

    # 将数据集转换为numpy数组
    x_train = np.stack([x.numpy() for x, _ in datasets[0]], axis=0)
    y_train = np.array([y.numpy() for _, y in datasets[0]])
    x_test = np.stack([x.numpy() for x, _ in datasets[1]], axis=0)
    y_test = np.array([y.numpy() for _, y in datasets[1]])

    # normalize
    x_train = x_train / 255.0
    x_test = x_test / 255.0

    return (x_train, y_train), (x_test, y_test)

In [4]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [5]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################

        self.W1 = tf.Variable(
            tf.random.normal([784, 256])
        )  # 784是输入层的大小，256是隐藏层的大小
        self.b1 = tf.Variable(tf.zeros([256]))
        self.W2 = tf.Variable(
            tf.random.normal([256, 10])
        )  # 256是隐藏层的大小，10是输出层的大小（对应10个类别）
        self.b2 = tf.Variable(tf.zeros([10]))

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################

        x = tf.reshape(x, [-1, 784])  # 将输入数据展平，这里-1为自动计算，算的是batch_size
        h = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)  # 计算隐藏层的输出, matmul是矩阵乘法
        logits = tf.matmul(h, self.W2) + self.b2  # 计算输出层的输出

        return logits

model = myModel()

optimizer = optimizers.adam_v2.Adam()

## 计算 loss

In [6]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [16]:
train_data, test_data = mnist_dataset()
print(train_data[0].shape, train_data[1].shape)
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

(60000, 28, 28, 1) (60000,)
epoch 0 : loss 68.07714 ; accuracy 0.0999
epoch 1 : loss 67.864815 ; accuracy 0.09985
epoch 2 : loss 67.6556 ; accuracy 0.09983333
epoch 3 : loss 67.449 ; accuracy 0.0999
epoch 4 : loss 67.24477 ; accuracy 0.0999
epoch 5 : loss 67.04279 ; accuracy 0.10001667
epoch 6 : loss 66.843 ; accuracy 0.100083336
epoch 7 : loss 66.64531 ; accuracy 0.10005
epoch 8 : loss 66.44969 ; accuracy 0.100216664
epoch 9 : loss 66.25607 ; accuracy 0.10036667
epoch 10 : loss 66.06442 ; accuracy 0.10035
epoch 11 : loss 65.874695 ; accuracy 0.100383334
epoch 12 : loss 65.68681 ; accuracy 0.100433335
epoch 13 : loss 65.5008 ; accuracy 0.1005
epoch 14 : loss 65.31666 ; accuracy 0.10053334
epoch 15 : loss 65.13432 ; accuracy 0.10046667
epoch 16 : loss 64.95373 ; accuracy 0.10065
epoch 17 : loss 64.774895 ; accuracy 0.10053334
epoch 18 : loss 64.59782 ; accuracy 0.10063333
epoch 19 : loss 64.42247 ; accuracy 0.10063333
epoch 20 : loss 64.24879 ; accuracy 0.10065
epoch 21 : loss 64.07674 